In [ ]:
from neuralprophet import NeuralProphet
from pytorch_lightning import LightningModule, Trainer
import torch
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(train, batch_size=64, shuffle=True)
validation_dataloader = DataLoader(validation, batch_size=64, shuffle=False)

In [ ]:
class NeuralProphetLightning(LightningModule, NeuralProphet):
    def __init__(self, train, validation_df, epochs, batch_size, learning_rate, early_stopping, metrics):
        super().__init__()
        self.train = train
        self.validation_df = validation_df
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.early_stopping = early_stopping
        self.metrics = metrics
        self.criterion = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def forward(self, x):
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        self.log('train_loss', loss)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.criterion(y_pred, y)
        self.log('val_loss', loss)
        return {'val_loss': loss}

    def configure_optimizers(self):
        return self.optimizer

In [ ]:
model = NeuralProphetLightning(train=train_dataloader, validation_df=validation_dataloader, 
epochs=1000, batch_size=64, learning_rate=1e-3, early_stopping=True, metrics=True)
trainer = Trainer(accelerator='mps', devices=1)
fitted = trainer.fit(model=model)
forecast_neuralprohet = fitted.predict(test)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8, svd_solver='auto').fit(features)
pca_features = pca.transform(features)
train_features, test_features = train_test_split(pca_features, test_size = 0.1)